In [ ]:
pip install transformers datasets wandb rouge_score evaluate

In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [3]:
from transformers import pipeline, AutoModelWithLMHead, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, TextDataset, DataCollatorForLanguageModeling
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import evaluate

# **Loading and Preprocessing the dataset**

In [ ]:
#load
df = pd.read_csv('create_data.csv')
#combine input and output sentence pairs
df['combined'] = '<s>' + df.input + '</s>' + '>>>>' + '<p>' + df.output + '</p>'
#train test split
train, valid = train_test_split(df['combined'],test_size=0.2)
train.to_csv('train.txt', sep ='\n', index = False)
valid.to_csv('valid.txt', sep ='\n', index = False)

In [4]:
#load
df = pd.read_csv('new_create_data.csv')
#combine input and output sentence pairs
df['combined'] = '<s>' + df.input + '</s>' + '>>>>' + '<p>' + df.output + '</p>'
#train test split
train, valid = train_test_split(df['combined'],test_size=0.2)
train.to_csv('train_new.txt', sep ='\n', index = False)
valid.to_csv('valid_new.txt', sep ='\n', index = False)

# **Train the model (gpt2)**

In [ ]:
def modeltrainer (text_path , valid_path, epochs , bs = 8, lr = 2e-4):

  model = AutoModelWithLMHead.from_pretrained('gpt2-medium')
  tokenizer = AutoTokenizer.from_pretrained('gpt2-medium')
  data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm = False)

  train_dataset = TextDataset(tokenizer=tokenizer, file_path = text_path, block_size = 96)
  valid_dataset = TextDataset(tokenizer=tokenizer, file_path = valid_path, block_size = 96)

  training_args = TrainingArguments(
    output_dir = "gpt2-medium-new",
    evaluation_strategy = "epoch",
    learning_rate=lr,
    per_device_train_batch_size=bs,
    per_device_eval_batch_size=bs,
    num_train_epochs=epochs,
    report_to="wandb",
    push_to_hub=True
    )
  
  trainer = Trainer(
    model,
    training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator
    )
  
  trainer.train()
  return model, trainer

In [ ]:
model, trainer = modeltrainer ('/content/train_new.txt' ,'/content/valid_new.txt', 6 , bs = 8, lr = 8e-6)

/usr/local/lib/python3.8/dist-packages/transformers/models/auto/modeling_auto.py:1177: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (214069 > 1024). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.8/dist-packages/huggingface_hub/repository.py:725: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.12. Please create the repository first using `create_repo(..., exists_ok=True)`.
  warnings.warn(
Cloning https://huggingface.co/SWQ/gpt2-medium-new into local empty directory.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306:

Epoch,Training Loss,Validation Loss
1,No log,2.258449
2,2.366500,2.115095
3,2.366500,2.027261
4,2.019400,1.970500
5,2.019400,1.937633
6,1.881000,1.927412


***** Running Evaluation *****
  Num examples = 567
  Batch size = 8
Saving model checkpoint to gpt2-medium-new/checkpoint-500
Configuration saved in gpt2-medium-new/checkpoint-500/config.json
Model weights saved in gpt2-medium-new/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 567
  Batch size = 8
***** Running Evaluation *****
  Num examples = 567
  Batch size = 8
Saving model checkpoint to gpt2-medium-new/checkpoint-1000
Configuration saved in gpt2-medium-new/checkpoint-1000/config.json
Model weights saved in gpt2-medium-new/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 567
  Batch size = 8
***** Running Evaluation *****
  Num examples = 567
  Batch size = 8
Saving model checkpoint to gpt2-medium-new/checkpoint-1500
Configuration saved in gpt2-medium-new/checkpoint-1500/config.json
Model weights saved in gpt2-medium-new/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 567
  Batch s

In [ ]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout)

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to gpt2-medium-new
Configuration saved in gpt2-medium-new/config.json
Model weights saved in gpt2-medium-new/pytorch_model.bin
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.30k/1.35G [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/SWQ/gpt2-medium-new
   2d32d4d..ad5a72c  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/SWQ/gpt2-medium-new
   2d32d4d..ad5a72c  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Causal Language Modeling', 'type': 'text-generation'}}
To https://huggingface.co/SWQ/gpt2-medium-new
   ad5a72c..3b1ef67  main -> main

   ad5a72c..3b1ef67  main -> main



'https://huggingface.co/SWQ/gpt2-medium-new/commit/ad5a72c8d48e36e28b9134e5bff427846093953e'

# **Inference**

In [5]:
model_combine = AutoModelWithLMHead.from_pretrained('SWQ/gpt2-medium-combine')
tokenizer = AutoTokenizer.from_pretrained('gpt2')

/usr/local/lib/python3.8/dist-packages/transformers/models/auto/modeling_auto.py:1177: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Downloading:   0%|          | 0.00/967 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.44G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [6]:
model = AutoModelWithLMHead.from_pretrained('SWQ/gpt2-medium-new')
tokenizer = AutoTokenizer.from_pretrained('gpt2')

Downloading:   0%|          | 0.00/967 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.44G [00:00<?, ?B/s]

In [16]:
def grammar_correctressult(model, tokenizer, text, num_correct=3):
  input_ids = tokenizer("<s>"+text+"</s>>>>><p>", return_tensors="pt")
  outputs = model.generate(**input_ids, num_beams=num_correct, num_return_sequences=num_correct,pad_token_id = 50256, max_new_tokens =  len(input_ids.input_ids[0])-4)
  outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
  result = []
  for sent in outputs:
      result.append(sent.split('</s>>>>><p>')[1].split('</p>')[0])
  return result

In [18]:
grammar_correctressult(model, tokenizer, "He eated a apples", num_correct=3)

['He ate an apple.',
 'He enjoyed a delicious apple.',
 'He ate a delicious apple.']

# **Evalution**

In [ ]:
test = pd.read_csv("test_combine.csv")

In [19]:
test_new = pd.read_csv("test.csv")

## Rouge1 score

In [10]:
def grammar_correct(model, tokenizer, text, num_correct=3):
  input_ids = tokenizer("<s>"+text+"</s>>>>><p>", return_tensors="pt")
  outputs = model.generate(**input_ids, num_beams=num_correct, num_return_sequences=num_correct,pad_token_id = 50256, max_new_tokens =  len(input_ids.input_ids[0])-4)
  return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [20]:
def calulate_rouge(model, tokenizer, text):
  results = []
  label = []
  for sent,ref in zip(text["input"], text["output"]):
    result = grammar_correct(model, tokenizer, sent, num_correct=2)
    results.append(result[0].split('</s>>>>><p>')[1].split('</p>')[0])
    results.append(result[1].split('</s>>>>><p>')[1].split('</p>')[0])
    label.append(ref)
    label.append(ref)
  rouge = evaluate.load('rouge')
  rougescore = rouge.compute(predictions=results,references=label)
  return rougescore

In [ ]:
results = calulate_rouge(model_combine, tokenizer, test)

In [ ]:
results

{'rouge1': 0.8331005064341981,
 'rouge2': 0.6935922720330069,
 'rougeL': 0.8294017749867504,
 'rougeLsum': 0.8296801574382713}

In [ ]:
results_onnew = calulate_rouge(model_combine, tokenizer, test_new)

In [ ]:
results_onnew

{'rouge1': 0.830621008626001,
 'rouge2': 0.691095888123865,
 'rougeL': 0.8287226448279077,
 'rougeLsum': 0.8282699310513233}

In [ ]:
results_new = calulate_rouge(model, tokenizer, test)

In [ ]:
results_new

{'rouge1': 0.7903167326976399,
 'rouge2': 0.6373762738645727,
 'rougeL': 0.7828858828149305,
 'rougeLsum': 0.7831101199239259}

In [ ]:
results_new_onnew = calulate_rouge(model, tokenizer, test_new)

In [ ]:
results_new_onnew

{'rouge1': 0.8139807378066597,
 'rouge2': 0.6713800077389219,
 'rougeL': 0.8079962876125868,
 'rougeLsum': 0.8080664622122823}

In [21]:
model_results = {"gpt2-medium-new_data": [round(0.8139807378066597*100, 2), round(0.7903167326976399*100, 2)], 
          "gpt2-medium-combine_data": [round(0.830621008626001*100, 2), round(0.8331005064341981*100, 2)]}
df_result = pd.DataFrame(data=model_results, index=["rouge score on created data", "rouge score on combined data"])
df_result

,gpt2-medium-new_data,gpt2-medium-combine_data
rouge score on created data,81.40,83.06
rouge score on combined data,79.03,83.31


## acceptance rate

In [ ]:
def calulate_acceptance_rate(model, tokenizer, text):
  results = []
  for sent in text["input"]:
    result = grammar_correct(model, tokenizer, sent, num_correct=2)
    results.append(result)
  count = 0
  total = len(results) * 2
  for sent_list in results:
    for sent in sent_list:
      sent = sent.split('</s>>>>><p>')[1].split('</p>')[0]
      response = openai.Completion.create(
      model="text-davinci-003",
      prompt="Print \"True\" if this is grammatically correct otherwise \"False\":\n\n{}".format(sent),
      temperature=0,
      max_tokens=60,
      top_p=1,  
      frequency_penalty=0,
      presence_penalty=0)
      if response["choices"][0]["text"].strip() == "True":
        count += 1
  return count/total

In [ ]:
acc_test = calulate_acceptance_rate(model_combine, tokenizer, test_new)

In [ ]:
acc_test

0.7896341463414634

In [ ]:
acc_test_combine = calulate_acceptance_rate(model_combine, tokenizer, test)

In [ ]:
acc_test_combine

0.7253787878787878

In [ ]:
acc_test_new = calulate_acceptance_rate(model, tokenizer, test_new)

In [ ]:
acc_test_new

0.8170731707317073

In [ ]:
acc_test_combine_new = calulate_acceptance_rate(model, tokenizer, test)

In [ ]:
acc_test_combine_new

0.7481060606060606

In [22]:
model_results = {"gpt2-medium-new_data": [round(0.8170731707317073, 2), round(0.7481060606060606, 2)], 
          "gpt2-medium-combine_data": [round(0.7896341463414634, 2), round(0.7253787878787878, 2)]}
df_result = pd.DataFrame(data=model_results, index=["acceptance rate on created data", "acceptance rate on combined data"])
df_result

,gpt2-medium-new_data,gpt2-medium-combine_data
acceptance rate on created data,0.82,0.79
acceptance rate on combined data,0.75,0.73


## save results

In [ ]:
def show_result(model, tokenizer, text, num_sentence=2):
  text_copy = text.copy()
  text_copy.rename(columns={"input":"Original Sentence", "output":"Reference"}, inplace = True)
  results = []
  for sent in text["input"]:
    result = grammar_correct(model, tokenizer, sent, num_correct=num_sentence)
    result = [i.split('</s>>>>><p>')[1].split('</p>')[0] for i in result]
    results.append(result)
  text_copy["Model Suggestions"] = results
  return text_copy

In [ ]:
df_show = show_result(model_combine, tokenizer, test, num_sentence=2)

In [ ]:
df_show.to_csv("model_result.csv")